In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from skimage.color import rgb2lab, lab2rgb
from tensorflow.keras.preprocessing.image import img_to_array, load_img

In [2]:
# 
LOAD_PATH = 'model_final/0402_3kimg_8-e04'
SAVED_MODEL_PATH = 'model_final/0402_3kimg_8-e04_w/Signature'
new_model = tf.keras.models.load_model(LOAD_PATH)

tf.saved_model.save(
    new_model, SAVED_MODEL_PATH,
    signatures={'my_signature':new_model.add.get_concrete_function()})

# # Convert the model to TF.lite
# converter = tf.lite.TFLiteConverter.from_saved_model("model_final/0402_3kimg_8-e04") # path to the SavedModel directory
# tflite_model = converter.convert()

# # Save the model.
# with open('model.tflite', 'wb') as f:
#   f.write(tflite_model)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-05-13 13:13:00.616619: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-05-13 13:13:00.616879: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


AttributeError: 'Functional' object has no attribute 'add'

In [ ]:
TFLITE_FILE_PATH = './model.tflite'
interpreter = tf.lite.Interpreter(TFLITE_FILE_PATH)
interpreter.allocate_tensors()

In [ ]:
signatures = 

In [ ]:
INPUT_IMG = interpreter.get_input_details()
OUTPUT_IMG = interpreter.get_output_details()

In [ ]:
imgs = []
for i in range(20):
    imgs.append(img_to_array(load_img("./dataset/color/" + str(i) + ".jpg", target_size=(256, 256))))
# imgs.append(img_to_array(load_img("./dataset/DiifGray/WechatIMG65.jpg", target_size=(256, 256))))
imgs = np.array(imgs, dtype=np.float32) / 255.0
labs = np.array([rgb2lab(rgb) for rgb in imgs])
input_shape = labs[:, :, :, 0]
input_shape = input_shape.reshape(input_shape.shape + (1, ))

In [ ]:
input_data = input_shape
input_data.shape

(20, 256, 256, 1)

In [ ]:
interpreter.set_tensor(INPUT_IMG[0]['index'], input_data)
# interpreter.set_tensor(INPUT_IMG[0]['index'], input_data)

ValueError: Cannot set tensor: Dimension mismatch. Got 20 but expected 1 for dimension 0 of input 0.

In [ ]:
interpreter.invoke()

In [ ]:
output_data = interpreter.get_tensor(OUTPUT_IMG[0]['index'])
print(output_data)

In [ ]:
y_pred_ab = output_data * 128.0
y_pred_lab = np.zeros(labs.shape)
for i in range(len(input_data)):
    y_pred_lab[i, :, :, 0] = input_data[i].reshape(input_data.shape[1:3])
    # y_pred_lab[i, :, :, 0] = x_train[i].reshape(x_test.shape[1:3])
    y_pred_lab[i, :, :, 1:] = y_pred_ab[i]

In [ ]:
y_pred_rgb = [lab2rgb(lab) for lab in y_pred_lab]
# testSet_rgb = [lab2rgb(lab) for lab in labs]
testSet_rgb = imgs
y_pred_rgb

In [ ]:
count = 1
n_sample = input_data.shape[0]
fig = plt.figure(figsize=(9, 3 * n_sample))
def extract_single_dim_from_LAB_convert_to_RGB(image, idim):
    z = np.zeros(image.shape)
    if idim != 0 : z[:, :, 0] = 80 # Brightness
    z[:, :, idim] = image[:, :, idim]
    z = lab2rgb(z)
    return z
for test_rgb, pred_rgb, test_lab in zip(testSet_rgb, y_pred_rgb, labs):

    ax = fig.add_subplot(n_sample, 3, count)
    lab_rgb_gray = extract_single_dim_from_LAB_convert_to_RGB(test_lab, 0) 
    ax.imshow(lab_rgb_gray)
    ax.axis("off")
    ax.set_title("Input")
    count += 1
    
    ax = fig.add_subplot(n_sample, 3, count)
    ax.imshow(test_rgb)
    ax.axis("off")
    ax.set_title("Ground Truth")
    count += 1

    ax = fig.add_subplot(n_sample, 3, count)
    ax.imshow(pred_rgb)
    ax.axis("off")
    ax.set_title("Predict")
    count += 1
plt.show()